In [ ]:
import joblib
import os
import torch
import torch.nn as nn
import pandas as pd
import ast
from torchtext.vocab import vocab
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import PreTrainedModel, PretrainedConfig
from functools import partial
from types import SimpleNamespace as sn
import torch.nn.functional as F
from torchinfo import summary
import random
import numpy as np
from pathlib import Path
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
from functools import partial
from ast import literal_eval
from torchmetrics import HammingDistance

In [ ]:
# if 'google.colab' in str(get_ipython()):
#   from google.colab import drive
#   drive.mount('/content/drive')
#   os.chdir('/content/drive/MyDrive/Colab_Notebooks/NLP/Assignment-4')

In [ ]:
df= joblib.load("/Users/anuraggupta/Library/CloudStorage/OneDrive-TheUniversityofTexasatDallas/UTD/Academics/Fall'23/NLP/Assignment/Assignment-4/df_multilabel_hw_cleaned.joblib")

In [ ]:
df.head()

,cleaned_text,Tags,Tag_Number
0,asp query stre dropdown webpage follow control...,c# asp.net,"[0, 9]"
1,run javascript code server java code want run ...,java javascript,"[1, 3]"
2,linq sql throw exception row find change hi li...,c# asp.net,"[0, 9]"
3,run python script php server run nginx web ser...,php python,"[2, 7]"
4,advice write function m try write function res...,javascript jquery,"[3, 5]"


In [ ]:
df.Tags.value_counts(normalize = True)

javascript jquery                   0.421469
c# asp.net                          0.168385
java android                        0.145951
php javascript                      0.045649
php jquery                          0.031606
                                      ...   
c# php javascript asp.net jquery    0.000021
javascript asp.net android          0.000021
php javascript iphone               0.000021
java php iphone c++ android         0.000021
javascript php                      0.000021
Name: Tags, Length: 176, dtype: float64

In [ ]:
tuple(literal_eval(df.Tag_Number[0]))

(0, 9)

In [ ]:
df['Tag_Number'] = df.Tag_Number.apply(lambda x: tuple(literal_eval(str(x))))

In [ ]:
df.head()

,cleaned_text,Tags,Tag_Number
0,asp query stre dropdown webpage follow control...,c# asp.net,"(0, 9)"
1,run javascript code server java code want run ...,java javascript,"(1, 3)"
2,linq sql throw exception row find change hi li...,c# asp.net,"(0, 9)"
3,run python script php server run nginx web ser...,php python,"(2, 7)"
4,advice write function m try write function res...,javascript jquery,"(3, 5)"


#### One Hot encoding of target variable

In [ ]:
multi_lb = MultiLabelBinarizer()
multi_lb_fit = multi_lb.fit_transform(df.Tag_Number)

In [ ]:
multi_lb.classes_

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
multi_lb_fit

array([[1, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
df['Target_encoded'] = multi_lb_fit.tolist()

In [ ]:
df.head

<bound method NDFrame.head of                                             cleaned_text                Tags  \
0      asp query stre dropdown webpage follow control...          c# asp.net   
1      run javascript code server java code want run ...     java javascript   
2      linq sql throw exception row find change hi li...          c# asp.net   
3      run python script php server run nginx web ser...          php python   
4      advice write function m try write function res...   javascript jquery   
...                                                  ...                 ...   
47422  take value edittext put decimal point all- wor...        java android   
47423  listen phone state application nee liste phone...        java android   
47424  android ui thread thread task want access main...        java android   
47425  dynamic table row creation html javascript htm...  asp.net javascript   
47426  image preloader work ie follow code preload im...   javascript jquery   

      Tag

## Creating custom torch Dataset

In [ ]:
X = df["cleaned_text"]
y = df["Target_encoded"]

In [ ]:
#creating test and temporary train(will be used furtehr to split in vaildation and training dataset)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)
#splitting train and validation from temporary dataset
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size = 0.25, random_state=42)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, X, y):

        self.X = X
        self.y = y

    def __len__(self):

        return(len(self.X))

    def __getitem__(self, idx):

        text = self.X.iloc[idx]
        labels = self.y.iloc[idx]
        sample = (labels, text)

        return sample

In [ ]:

train_dataset = CustomDataset(X_train, y_train)
valid_dataset = CustomDataset(X_val, y_val)
test_dataset = CustomDataset(X_test, y_test)

In [ ]:
for i, (y, x) in enumerate(train_dataset):
    if i == 10:
        print(i, y, x)

10 [0, 0, 0, 1, 0, 1, 0, 0, 0, 0] load content good practice sort new javascript bug code like $ button').on('click function      var d = $ this).attr("id d file sans extension      $ .content').load(id + .php   work fine head tell bad practice thought


## Create Vocab

In [ ]:
def get_vocab(dataset, min_freq):
    """
    Generate a vocabulary from a dataset.

    Args:
        dataset (list of tuple): List of tuples where each tuple contains a label and a text.
        min_freq (int): The minimum frequency for a token to be included in the vocabulary.

    Returns:
        torchtext.vocab.Vocab: Vocabulary object.
    """
    # Initialize a counter object to hold token frequencies
    counter = Counter()

    # Update the counter with tokens from each text in the dataset
    for (l_, text) in dataset:
        counter.update(str(text).split())

    # Create a vocabulary using the counter object
    # Tokens that appear fewer times than `min_freq` are excluded
    my_vocab = vocab(counter, min_freq=min_freq)

    # Insert a '<unk>' token at index 0 to represent unknown words
    my_vocab.insert_token('<unk>', 0)

    # Set the default index to 0
    # This ensures that any unknown word will be mapped to '<unk>'
    my_vocab.set_default_index(0)

    return my_vocab

In [ ]:
df_vocab = get_vocab(train_dataset, 20)

In [ ]:
# Get the index where the frequency of the word is saved
df_vocab.get_stoi()

{'androidruntime(3338': 5676,
 '19:30:30.288': 5675,
 '17:55:40.935': 5674,
 'class="dot"></div': 5670,
 'mapview(32047': 5669,
 'system.err(1158': 5667,
 'androidruntime(353': 5664,
 'name="checkboxlist"</li': 5663,
 'androidruntime(309': 5661,
 '.next().slidedown': 5660,
 'androidruntime(27320': 5657,
 "d\\').e({f": 5654,
 'nextactionhref': 5653,
 'class="black_block"></div': 5652,
 'androidruntime(300': 5650,
 "profile').remove": 5648,
 'androidruntime(296': 5646,
 'omissing': 5644,
 '15:49:31.438': 5639,
 'insertparameter': 5636,
 'windowmanager(30601': 5635,
 'androidruntime(585': 5633,
 '13:44:28.154': 5632,
 'androidruntime(2027': 5626,
 '15:12:57.885': 5625,
 'androidruntime(861': 5624,
 '13:45:41.136': 5622,
 'rset': 5615,
 'pagehtml': 5614,
 'href="http://ee.dev': 5613,
 'sbprogress': 5612,
 'href="#">list': 5610,
 'kim': 5608,
 'class="bio': 5602,
 '01:45:06.726': 5598,
 'results.add(item_detail': 5596,
 '14:10:03.545': 5591,
 'androidruntime(28106': 5590,
 'androidruntime(6

# <Font color = 'pickle'>**Create DataLoader for Embedding**

In [ ]:
# Creating a function that will be used to get the indices of words from vocab
def text_pipeline(x, vocab):
    """Converts text to a list of indices using a vocabulary dictionary"""
    return [vocab[token] for token in str(x).split()]

In [ ]:
def collate_batch(batch, my_vocab):
    """
    Collates a batch of samples into tensors of labels, texts, and offsets.

    Parameters:
        batch (list): A list of tuples, each containing a label and a text.

    Returns:
        tuple: A tuple containing three tensors:
               - Labels tensor
               - Concatenated texts tensor
               - Offsets tensor indicating the start positions of each text in the concatenated tensor
    """
    # Unpack the batch into separate lists for labels and texts
    labels, texts = zip(*batch)

    # Convert the list of labels into a tensor of dtype int32
    labels = torch.tensor(labels, dtype=torch.long)

    # Convert the list of texts into a list of lists; each inner list contains the vocabulary indices for a text
    list_of_list_of_indices = [text_pipeline(text, my_vocab) for text in texts]

    # Compute the offsets for each text in the concatenated tensor
    offsets = [0] + [len(i) for i in list_of_list_of_indices]
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)

    # Concatenate all text indices into a single tensor
    texts = torch.cat([torch.tensor(i, dtype=torch.int64) for i in list_of_list_of_indices])

    return (texts, offsets), labels

In [ ]:
batch_size = 2
check_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           collate_fn=partial(collate_batch, my_vocab = df_vocab),
                                           )

torch.manual_seed(22)
for (text, offset), label in check_loader:
    print(text, offset, label)
    break

tensor([  21, 2398,  221,   77,   81,    0,  102, 1198,   44,    0,  102,   44,
         162,  221,  222,   21,  116,    0,   80,   81,    0, 1503,  313,   81,
         134,  106,   35,  173,   55,  106,    0,    0,  701,   55,  302,   83,
         173,   55,  106,    0,   77,   56,   80,    0,   55,  106,   80,    0,
          45,   55,  106,  158,  108]) tensor([ 0, 27]) tensor([[0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 1, 0, 0, 0, 0]])


## Model

In [ ]:
class CustomBlock(nn.Module):
    def __init__(self, input_dim, output_dim, drop_prob):

        super().__init__()

        self.layers = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.ReLU(),
            nn.Dropout(p=drop_prob),
            nn.BatchNorm1d(num_features=output_dim)
        )
    def forward(self, x):
      return self.layers(x)

In [ ]:
class CustomMLP(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim1, hidden_dim2, drop_prob1, drop_prob2, num_outputs):
        super().__init__()

        self.embedding_bag = nn.EmbeddingBag(vocab_size, embedding_dim)
        self.layers = nn.Sequential(
            CustomBlock(embedding_dim , hidden_dim1, drop_prob1),
            CustomBlock(hidden_dim1, hidden_dim2, drop_prob2),
            nn.Linear(hidden_dim2, num_outputs)
        )

    def forward(self, input_tuple):
        data, offsets = input_tuple
        embed_out = self.embedding_bag(data, offsets)
        out = self.layers(embed_out)

        return out

In [ ]:
# Define the device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Define the sequential model
model= CustomMLP(vocab_size= len(df_vocab), embedding_dim = 300, hidden_dim1 = 200, hidden_dim2 = 100, drop_prob1=0.5, drop_prob2=0.5, num_outputs=10)

# Move the model to the device
model = model.to(device)

# Generate some dummy input data and offsets, and move them to the device
data = torch.tensor([1, 2, 4, 5, 4], dtype = torch.int32).to(device)
offsets = torch.tensor([0, 2, 4], dtype = torch.int32).to(device)

# Generate summary
summary(model, input_data=[(data, offsets)], device=device, depth =4)

Layer (type:depth-idx)                   Output Shape              Param #
CustomMLP                                [3, 10]                   --
├─EmbeddingBag: 1-1                      [3, 300]                  1,703,100
├─Sequential: 1-2                        [3, 10]                   --
│    └─CustomBlock: 2-1                  [3, 200]                  --
│    │    └─Sequential: 3-1              [3, 200]                  --
│    │    │    └─Linear: 4-1             [3, 200]                  60,200
│    │    │    └─ReLU: 4-2               [3, 200]                  --
│    │    │    └─Dropout: 4-3            [3, 200]                  --
│    │    │    └─BatchNorm1d: 4-4        [3, 200]                  400
│    └─CustomBlock: 2-2                  [3, 100]                  --
│    │    └─Sequential: 3-2              [3, 100]                  --
│    │    │    └─Linear: 4-5             [3, 100]                  20,100
│    │    │    └─ReLU: 4-6               [3, 100]                  --

## Model training and validation

In [ ]:
def step(inputs, targets, model, device, loss_function=None, optimizer=None):

    # Move the model and data to the device
    model = model.to(device)
    inputs = tuple(input_tensor.to(device) for input_tensor in inputs)

    targets = targets.to(device, dtype=torch.float32)

    # Step 1: Forward pass to get the model's predictions
    outputs = model(inputs)

    # Step 2: Compute the loss using the provided loss function
    if loss_function:
        loss = loss_function(outputs, targets)

    # Step 2: Calculate the number of correctly classified samples
    predicted = (outputs > 0.5).float()
    #correct = (predicted == targets).sum().item()

    # Step 3 and 4: Perform backward pass and update model parameters if an optimizer is provided
    if optimizer:
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_value_(model.parameters(), clip_value=10.0)
        optimizer.step()


    # Return relevant metrics
    if loss_function:
        return loss, outputs, predicted
    else:
        return outputs, predicted

In [ ]:
def train_epoch(train_loader, model, device, loss_function, optimizer):

    train_hamming_distance = HammingDistance(task = "multilabel", num_labels = 10).to(device)

    # Set the model to training mode
    model.train()

    # Initialize variables to track running training loss and correct predictions
    running_train_loss = 0.0

    # Iterate over all batches in the training data
    for inputs, targets in train_loader:
        # Perform a forward and backward pass, updating model parameters
        loss, _, predict = step(inputs, targets, model, device, loss_function, optimizer)
        # ham_dist = train_hamming_distance(predict, targets)

        # Update running loss and correct predictions counter
        running_train_loss += loss.item()
        train_hamming_distance.update(predict, targets)

    epoch_hamming_distance = train_hamming_distance.compute()

    # Compute average loss and accuracy for the entire training set
    train_loss = running_train_loss / len(train_loader)
    print(f'Train Loss: {train_loss:.4f} | Train Hamming Distance: {epoch_hamming_distance:.4f}')

    train_hamming_distance.reset()

    return train_loss, epoch_hamming_distance

In [ ]:
def val_epoch(valid_loader, model, device, loss_function):

    """
    Validates the model for one epoch using the provided data loader.

    Parameters:
    - valid_loader (torch.utils.data.DataLoader): DataLoader object for the validation set.
    - model (torch.nn.Module): The neural network model to be validated.
    - device (torch.device): The computing device (CPU or GPU).
    - loss_function (torch.nn.Module): The loss function to evaluate the model.

    Returns:
    - val_loss (float): Average validation loss for the epoch.
    - val_acc (float): Validation accuracy for the epoch.
    """
    train_hamming_distance = HammingDistance(task = "multilabel", num_labels = 10).to(device)
    # Set the model to evaluation mode

    model.eval()

    # Initialize variables to track running validation loss and correct predictions
    running_val_loss = 0.0

    for inputs, targets in valid_loader:
        # Perform a forward and backward pass, updating model parameters
        loss, _, predict = step(inputs, targets, model, device, loss_function, optimizer = None)
        # train_hamming_distance = train_hamming_distance(predict, targets)

        # Update running loss and correct predictions counter
        running_val_loss += loss.item()
        train_hamming_distance.update(predict, targets)

    epoch_hamming_distance = train_hamming_distance.compute()

    # Compute average loss and accuracy for the entire training set
    train_loss = running_val_loss / len(valid_loader)
    print(f'Valid Loss: {train_loss:.4f} | Valid Hamming Distance: {epoch_hamming_distance:.4f}')

    train_hamming_distance.reset()

    return train_loss, epoch_hamming_distance

In [ ]:
def train(train_loader, valid_loader, model, optimizer, loss_function, epochs, device):

    # Initialize lists to store metrics for each epoch
    train_loss_history = []
    valid_loss_history = []
    train_ham_history = []
    valid_ham_history = []

    # Loop over the number of specified epochs
    for epoch in range(epochs):
        # Train model on training data and capture metrics
        train_loss, train_ham = train_epoch(train_loader, model, device, loss_function, optimizer)

        # Validate model on validation data and capture metrics
        valid_loss, valid_ham = val_epoch(valid_loader, model, device, loss_function)

        # Store metrics for this epoch
        train_loss_history.append(train_loss)
        train_ham_history.append(train_ham)
        valid_loss_history.append(valid_loss)
        valid_ham_history.append(valid_ham)

        # Output epoch-level summary
        print(f"Epoch {epoch+1}/{epochs}")
        print(f"Train Loss: {train_loss:.4f} | Train Hamming distance: {train_ham*100:.2f}%")
        print(f"Valid Loss: {valid_loss:.4f} | Valid Hamming distance: {valid_ham*100:.2f}%")
        print()

    return train_loss_history, train_ham_history, valid_loss_history, valid_ham_history

# <Font color = 'pickle'>**Hyperparameters**

In [ ]:
hyperparameters = sn(
    # model Parameters
    EMBED_DIM=300,
    VOCAB_SIZE=len(df_vocab),
    OUTPUT_DIM=10,
    HIDDEN_DIM1=200,
    HIDDEN_DIM2=100,
    DROP_PROB1=0.5,
    DROP_PROB2=0.5,
    NUM_OUTPUTS=10,

    # training
    EPOCHS=5,
    BATCH_SIZE=128,
    LEARNING_RATE=0.001,
    WEIGHT_DECAY=0.1,
)
collate_fn = partial(collate_batch, my_vocab = df_vocab)

# Model function and training

In [ ]:
# Fix seed value
SEED = 2345
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

collate_fn = partial(collate_batch, my_vocab = df_vocab)

# Data Loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=hyperparameters.BATCH_SIZE, shuffle=True,
                                           collate_fn = collate_fn, num_workers=4)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=hyperparameters.BATCH_SIZE, shuffle=False,
                                           collate_fn=collate_fn,  num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=hyperparameters.BATCH_SIZE,  shuffle=False,
                                          collate_fn=collate_fn,  num_workers=4)


loss_function = nn.BCEWithLogitsLoss()

model_as4 = CustomMLP(vocab_size=hyperparameters.VOCAB_SIZE,
                       embedding_dim=hyperparameters.EMBED_DIM,
                       hidden_dim1=hyperparameters.HIDDEN_DIM1,
                       hidden_dim2=hyperparameters.HIDDEN_DIM2,
                       drop_prob1=hyperparameters.DROP_PROB1,
                       drop_prob2=hyperparameters.DROP_PROB2,
                       num_outputs=hyperparameters.NUM_OUTPUTS)

def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.kaiming_normal_(m.weight)
        torch.nn.init.zeros_(m.bias)


# apply initialization recursively  to all modules
model_as4.apply(init_weights)

optimizer = torch.optim.SGD(model.parameters(), lr = hyperparameters.LEARNING_RATE)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
##Sanity Check
for input_ , targets  in train_loader:

    # move inputs and outputs to GPUs
    model_as4 = model_as4.to(device)

    input_ = tuple(input_tensor.to(device) for input_tensor in input_)
    targets = targets.to(device, dtype = torch.float32)

    model_as4.eval()
    # Forward pass
    output = model_as4(input_)
    loss = loss_function(output, targets)
    print(f'Actual loss: {loss}')
    break

print(f'Expected Theoretical loss: {np.log(2)}')

## model training

In [ ]:
train_losses, train_ham_d, valid_losses, valid_ham_d = train(train_loader, valid_loader, model_as4, optimizer, loss_function, hyperparameters.EPOCHS, device)

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/anuraggupta/anaconda3/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/anuraggupta/anaconda3/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'CustomDataset' on <module '__main__' (built-in)>
